## Xception model training

### import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from keras import optimizers
from keras import regularizers
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [3]:
nb_classes = 21
nb_cpus = cpu_count()//2
nb_gpus = 2

image_size = (299, 299)
input_shape= (299,299,3)

### train model

In [4]:
from keras import optimizers
from keras import regularizers

with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    # m_out = SeparableConv2D(4096, kernel_size=3, strides=1)(m_out)
    # m_out = BatchNormalization()(m_out)
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    model = Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [5]:
parallel_model.load_weights('xcp-21cla_release/batch6.4_007_0.0655.hdf5')

In [6]:
model.save_weights("xcp-21cla_release/batch6.4_train3_finetuned.h5")